# Numerical Integration Techniques

This notebook demonstrates various numerical integration methods in rustlab-numerical, including quadrature rules and adaptive algorithms.

## Setup and Dependencies

In [2]:
:dep rustlab-math = { path = "../../rustlab-math" }
:dep rustlab-numerical = { path = ".." }
:dep rustlab-plotting = { path = "../../rustlab-plotting" }

use rustlab_math::{VectorF64, vec64, range, FunctionalMap};
use rustlab_numerical::integration::*;
use rustlab_plotting::*;
use std::f64::consts::PI;

## 1. Trapezoidal Rule

The trapezoidal rule is the simplest integration method with O(h²) error.

In [3]:
{
    // Integrate sin(x) from 0 to π (exact answer = 2)
    let f = |x: f64| x.sin();
    let intervals = vec![10, 50, 100, 500, 1000];
    
    println!("Trapezoidal Rule: ∫₀^π sin(x) dx (exact = 2.0)");
    println!("{:<10} {:<15} {:<15} {:<15}", "Intervals", "Result", "Error", "Error Ratio");
    
    let mut prev_error = 0.0;
    for n in intervals {
        let result = trapz(f, 0.0, PI, n)?;
        let error = (result - 2.0).abs();
        let ratio = if prev_error > 0.0 { prev_error / error } else { 0.0 };
        
        println!("{:<10} {:<15.8} {:<15.2e} {:<15.2}", n, result, error, ratio);
        prev_error = error;
    }
}

Trapezoidal Rule: ∫₀^π sin(x) dx (exact = 2.0)
Intervals  Result          Error           Error Ratio    
10         1.98352354      1.65e-2         0.00           
50         1.99934198      6.58e-4         25.04          
100        1.99983550      1.64e-4         4.00           
500        1.99999342      6.58e-6         25.00          
1000       1.99999836      1.64e-6         4.00           


()

## 2. Simpson's Rule

Simpson's rule provides O(h⁴) accuracy using parabolic approximation.

In [4]:
{
    // Compare Simpson's rule with trapezoidal
    let f = |x: f64| x.sin();
    let intervals = vec![10, 50, 100, 500, 1000];
    
    println!("Simpson's Rule: ∫₀^π sin(x) dx (exact = 2.0)");
    println!("{:<10} {:<15} {:<15} {:<15}", "Intervals", "Result", "Error", "Error Ratio");
    
    let mut prev_error = 0.0;
    for n in intervals {
        let result = simpson(f, 0.0, PI, n)?;
        let error = (result - 2.0).abs();
        let ratio = if prev_error > 0.0 { prev_error / error } else { 0.0 };
        
        println!("{:<10} {:<15.8} {:<15.2e} {:<15.2}", n, result, error, ratio);
        prev_error = error;
    }
}

Simpson's Rule: ∫₀^π sin(x) dx (exact = 2.0)
Intervals  Result          Error           Error Ratio    
10         2.00010952      1.10e-4         0.00           
50         2.00000017      1.73e-7         632.12         
100        2.00000001      1.08e-8         16.01          
500        2.00000000      1.73e-11        625.10         
1000       2.00000000      1.08e-12        16.05          


()

## 3. Romberg Integration

Romberg integration uses Richardson extrapolation for adaptive high-precision integration.

In [5]:
{
    // Test Romberg integration with different tolerances
    let f = |x: f64| x.sin();
    let tolerances = vec![1e-4, 1e-6, 1e-8, 1e-10, 1e-12];
    
    println!("Romberg Integration: ∫₀^π sin(x) dx (exact = 2.0)");
    println!("{:<12} {:<15} {:<15}", "Tolerance", "Result", "Error");
    
    for tol in tolerances {
        let result = romberg(f, 0.0, PI, tol, 20)?; // Add max_iterations parameter
        let error = (result - 2.0).abs();
        
        println!("{:<12.0e} {:<15.12} {:<15.2e}", tol, result, error);
    }
}

Romberg Integration: ∫₀^π sin(x) dx (exact = 2.0)
Tolerance    Result          Error          
1e-4         1.999999994587  5.41e-9        
1e-6         2.000000000001  1.32e-12       
1e-8         2.000000000001  1.32e-12       
1e-10        2.000000000000  4.44e-16       
1e-12        2.000000000000  4.44e-16       


()

## 4. Integration of Different Function Types

Test various functions with known analytical solutions.

In [6]:
{
    // Test polynomial integration: x³ from 0 to 2 (exact = 4)
    let poly = |x: f64| x.powi(3);
    let poly_result = simpson(poly, 0.0, 2.0, 100)?;
    let poly_error = (poly_result - 4.0).abs();
    
    // Test exponential: e^x from 0 to 1 (exact = e - 1)
    let exp_func = |x: f64| x.exp();
    let exp_result = simpson(exp_func, 0.0, 1.0, 100)?;
    let exp_exact = 1.0_f64.exp() - 1.0;
    let exp_error = (exp_result - exp_exact).abs();
    
    // Test logarithm: ln(x) from 1 to e (exact = 1)
    let log_func = |x: f64| x.ln();
    let log_result = simpson(log_func, 1.0, 1.0_f64.exp(), 100)?;
    let log_error = (log_result - 1.0).abs();
    
    println!("Integration of Various Functions:");
    println!("∫₀² x³ dx = {:.8} (exact = 4.0, error = {:.2e})", poly_result, poly_error);
    println!("∫₀¹ eˣ dx = {:.8} (exact = {:.8}, error = {:.2e})", exp_result, exp_exact, exp_error);
    println!("∫₁ᵉ ln(x) dx = {:.8} (exact = 1.0, error = {:.2e})", log_result, log_error);
}

Integration of Various Functions:
∫₀² x³ dx = 4.00000000 (exact = 4.0, error = 8.88e-16)
∫₀¹ eˣ dx = 1.71828183 (exact = 1.71828183, error = 9.55e-11)
∫₁ᵉ ln(x) dx = 1.00000000 (exact = 1.0, error = 9.20e-10)


()

## 5. Oscillatory Functions

Integration of rapidly oscillating functions poses special challenges.

In [7]:
{
    // High frequency sine function: sin(20x) from 0 to π
    let high_freq = |x: f64| (20.0 * x).sin();
    let exact = 0.0; // sin(20x) integrates to 0 over full period
    
    let intervals = vec![100, 500, 1000, 2000, 5000];
    
    println!("Oscillatory Function: ∫₀^π sin(20x) dx (exact ≈ 0.0)");
    println!("{:<10} {:<15} {:<15} {:<15}", "Method", "Intervals", "Result", "Error");
    
    for n in intervals {
        let trap_result = trapz(high_freq, 0.0, PI, n)?;
        let simp_result = simpson(high_freq, 0.0, PI, n)?;
        
        let trap_error = (trap_result - exact).abs();
        let simp_error = (simp_result - exact).abs();
        
        println!("{:<10} {:<15} {:<15.8} {:<15.2e}", "Trapz", n, trap_result, trap_error);
        println!("{:<10} {:<15} {:<15.8} {:<15.2e}", "Simpson", n, simp_result, simp_error);
    }
}

Oscillatory Function: ∫₀^π sin(20x) dx (exact ≈ 0.0)
Method     Intervals       Result          Error          
Trapz      100             -0.00000000     1.16e-15       
Simpson    100             -0.00000000     1.05e-15       
Trapz      500             0.00000000      1.48e-16       
Simpson    500             0.00000000      2.02e-16       
Trapz      1000            0.00000000      1.88e-16       
Simpson    1000            0.00000000      6.42e-17       
Trapz      2000            0.00000000      1.30e-16       
Simpson    2000            0.00000000      2.55e-17       
Trapz      5000            0.00000000      2.95e-17       
Simpson    5000            -0.00000000     2.86e-16       


()

## 6. Convergence Visualization

Plot convergence behavior of different integration methods.

In [8]:
{
    // Generate convergence data using ergonomic operations
    let f = |x: f64| x.exp(); // e^x from 0 to 1
    let exact = 1.0_f64.exp() - 1.0;
    
    // Create intervals using ergonomic range
    let intervals_raw: Vec<usize> = (1..=50).map(|i| i * 20).collect();
    let intervals_vec = vec64![]; // Empty vector to collect errors
    
    let mut trap_errors_vec = Vec::new();
    let mut simp_errors_vec = Vec::new();
    let mut intervals_f64_vec = Vec::new();
    
    for &n in &intervals_raw {
        let trap_result = trapz(f, 0.0, 1.0, n)?;
        let simp_result = simpson(f, 0.0, 1.0, n)?;
        
        trap_errors_vec.push((trap_result - exact).abs());
        simp_errors_vec.push((simp_result - exact).abs());
        intervals_f64_vec.push(n as f64);
    }
    
    let intervals_f64 = VectorF64::from_slice(&intervals_f64_vec);
    let trap_errors = VectorF64::from_slice(&trap_errors_vec);
    let simp_errors = VectorF64::from_slice(&simp_errors_vec);
    
    // Plot convergence using correct plotting API
    Plot::new()
        .line_with(&intervals_f64, &trap_errors, "Trapezoidal Rule")
        .line_with(&intervals_f64, &simp_errors, "Simpson's Rule")
        .title("Integration Error Convergence")
        .xlabel("Number of Intervals")
        .ylabel("Absolute Error")
        .yscale(Scale::Log10)
        .legend(true)
        .grid(true)
        .show()?;
    
    println!("Final errors at 1000 intervals:");
    println!("Trapezoidal: {:.2e}", trap_errors.get(trap_errors.len()-1).unwrap());
    println!("Simpson: {:.2e}", simp_errors.get(simp_errors.len()-1).unwrap());
}

Final errors at 1000 intervals:
Trapezoidal: 1.43e-7
Simpson: 7.55e-15


()

## 7. Performance Benchmarking

Compare computational efficiency of different methods.

In [9]:
{
    use std::time::Instant;
    
    let f = |x: f64| (x * x + 1.0).sqrt(); // Moderately complex function
    let n_intervals = 10000;
    let n_runs = 1000;
    
    println!("Performance benchmark ({} runs, {} intervals):", n_runs, n_intervals);
    
    // Trapezoidal rule benchmark
    let start = Instant::now();
    for _ in 0..n_runs {
        let _ = trapz(f, 0.0, 1.0, n_intervals);
    }
    let trap_time = start.elapsed();
    
    // Simpson's rule benchmark
    let start = Instant::now();
    for _ in 0..n_runs {
        let _ = simpson(f, 0.0, 1.0, n_intervals);
    }
    let simp_time = start.elapsed();
    
    // Romberg integration benchmark (fewer runs due to complexity)
    let romberg_runs = 100;
    let start = Instant::now();
    for _ in 0..romberg_runs {
        let _ = romberg(f, 0.0, 1.0, 1e-8, 20); // Add max_iterations parameter
    }
    let romberg_time = start.elapsed();
    
    println!("Trapezoidal: {:.2?} ({:.1} μs/call)", 
             trap_time, trap_time.as_micros() as f64 / n_runs as f64);
    println!("Simpson: {:.2?} ({:.1} μs/call)", 
             simp_time, simp_time.as_micros() as f64 / n_runs as f64);
    println!("Romberg: {:.2?} ({:.1} μs/call)", 
             romberg_time, romberg_time.as_micros() as f64 / romberg_runs as f64);
}

Performance benchmark (1000 runs, 10000 intervals):
Trapezoidal: 18.25ms (18.2 μs/call)
Simpson: 18.89ms (18.9 μs/call)
Romberg: 108.00µs (1.1 μs/call)


()

## 8. Advanced Application: Area Under Curve

Practical example computing areas and comparing different approximation methods.

In [10]:
{
    // Define a complex curve: Gaussian bell curve
    let gaussian = |x: f64| (-0.5 * x * x).exp() / (2.0 * PI).sqrt();
    
    // Compute areas for different ranges (should approach 1.0 for full range)
    let ranges = vec![
        (-1.0, 1.0),   // ±1σ ≈ 68%
        (-2.0, 2.0),   // ±2σ ≈ 95%
        (-3.0, 3.0),   // ±3σ ≈ 99.7%
        (-4.0, 4.0),   // ±4σ ≈ 99.99%
    ];
    
    println!("Gaussian Distribution Area Calculation:");
    println!("{:<15} {:<15} {:<15} {:<15}", "Range", "Trapz", "Simpson", "Romberg");
    
    for (a, b) in ranges {
        let trap_area = trapz(gaussian, a, b, 1000)?;
        let simp_area = simpson(gaussian, a, b, 1000)?;
        let romberg_area = romberg(gaussian, a, b, 1e-8, 20)?; // Add max_iterations parameter
        
        let range_str = format!("[{:.1}, {:.1}]", a, b);
        println!("{:<15} {:<15.6} {:<15.6} {:<15.6}", 
                 range_str, trap_area, simp_area, romberg_area);
    }
}

Gaussian Distribution Area Calculation:
Range           Trapz           Simpson         Romberg        
[-1.0, 1.0]     0.682689        0.682689        0.682689       
[-2.0, 2.0]     0.954499        0.954500        0.954500       
[-3.0, 3.0]     0.997300        0.997300        0.997300       
[-4.0, 4.0]     0.999937        0.999937        0.999937       


()

In [11]:
{
    // Visualize the Gaussian and integration approximations using range macro
    let gaussian = |x: f64| (-0.5 * x * x).exp() / (2.0 * PI).sqrt();
    
    let x = range!(-4.0 => 4.0, 200);
    let y = (&x).map(|xi| gaussian(*xi));
    
    // Create trapezoidal approximation with coarse intervals
    let x_coarse = range!(-3.0 => 3.0, 13);
    let y_coarse = (&x_coarse).map(|xi| gaussian(*xi));
    
    Plot::new()
        .line_with(&x, &y, "Gaussian PDF")
        .line_with(&x_coarse, &y_coarse, "Trapezoidal Approx")
        .scatter_with(&x_coarse, &y_coarse, "Sample Points")
        .title("Gaussian Distribution and Trapezoidal Approximation")
        .xlabel("x")
        .ylabel("Probability Density")
        .legend(true)
        .grid(true)
        .show()?;
    
    // Calculate the approximation error
    let exact_area = romberg(gaussian, -3.0, 3.0, 1e-10, 20)?; // Add max_iterations parameter
    let approx_area = trapz(gaussian, -3.0, 3.0, 12)?;
    let error = (exact_area - approx_area).abs();
    
    println!("Area [-3,3]: exact = {:.8}, approx = {:.8}, error = {:.2e}", 
             exact_area, approx_area, error);
}

Error: type `f64` cannot be dereferenced

Error: type `f64` cannot be dereferenced

## 9. Error Analysis and Adaptive Strategies

Understanding when to use different methods based on function characteristics.

In [12]:
{
    // Test functions with different smoothness properties
    println!("Method Comparison for Different Function Types:");
    println!("{:<20} {:<12} {:<12} {:<12}", "Function", "Trapz Error", "Simp Error", "Efficiency");
    
    // Test each function separately to avoid closure type issues
    let n = 100;
    
    // Smooth function: sin(x)
    let f1 = |x: f64| x.sin();
    let exact1 = 2.0;
    let trap1 = trapz(f1, 0.0, PI, n)?;
    let simp1 = simpson(f1, 0.0, PI, n)?;
    let trap_err1 = (trap1 - exact1).abs();
    let simp_err1 = (simp1 - exact1).abs();
    let eff1 = trap_err1 / simp_err1;
    println!("{:<20} {:<12.2e} {:<12.2e} {:<12.1}", "Smooth: sin(x)", trap_err1, simp_err1, eff1);
    
    // Polynomial: x⁴
    let f2 = |x: f64| x.powi(4);
    let exact2 = 0.2;
    let trap2 = trapz(f2, 0.0, 1.0, n)?;
    let simp2 = simpson(f2, 0.0, 1.0, n)?;
    let trap_err2 = (trap2 - exact2).abs();
    let simp_err2 = (simp2 - exact2).abs();
    let eff2 = trap_err2 / simp_err2;
    println!("{:<20} {:<12.2e} {:<12.2e} {:<12.1}", "Polynomial: x⁴", trap_err2, simp_err2, eff2);
    
    // Exponential: e^x
    let f3 = |x: f64| x.exp();
    let exact3 = 1.0_f64.exp() - 1.0;
    let trap3 = trapz(f3, 0.0, 1.0, n)?;
    let simp3 = simpson(f3, 0.0, 1.0, n)?;
    let trap_err3 = (trap3 - exact3).abs();
    let simp_err3 = (simp3 - exact3).abs();
    let eff3 = trap_err3 / simp_err3;
    println!("{:<20} {:<12.2e} {:<12.2e} {:<12.1}", "Exponential: e^x", trap_err3, simp_err3, eff3);
    
    // Sharp peak: e^(-x²)
    let f4 = |x: f64| (-x*x).exp();
    let exact4 = PI.sqrt();
    let trap4 = trapz(f4, -2.0, 2.0, n)?;
    let simp4 = simpson(f4, -2.0, 2.0, n)?;
    let trap_err4 = (trap4 - exact4).abs();
    let simp_err4 = (simp4 - exact4).abs();
    let eff4 = trap_err4 / simp_err4;
    println!("{:<20} {:<12.2e} {:<12.2e} {:<12.1}", "Sharp peak: e^(-x²)", trap_err4, simp_err4, eff4);
    
    println!("\nEfficiency > 1 means Simpson's rule is better");
}

Method Comparison for Different Function Types:
Function             Trapz Error  Simp Error   Efficiency  
Smooth: sin(x)       1.64e-4      1.08e-8      15196.6     
Polynomial: x⁴       3.33e-5      1.33e-9      24999.8     
Exponential: e^x     1.43e-5      9.55e-11     150001.2    
Sharp peak: e^(-x²)  8.31e-3      8.29e-3      1.0         

Efficiency > 1 means Simpson's rule is better


()

## 10. Practical Integration Tips

Guidelines for choosing the right integration method.

In [13]:
{
    println!("Integration Method Selection Guide:");
    println!();
    
    // Demonstrate computational cost vs accuracy trade-off
    let f = |x: f64| (x * x + 1.0).sqrt();
    let exact = 0.5 * (2.0_f64.sqrt() + (1.0 + 2.0_f64.sqrt()).ln()); // Analytical result
    
    println!("{:<20} {:<15} {:<15} {:<15}", "Method", "Result", "Error", "Relative Error");
    
    // Test each method separately to avoid closure type issues
    let trap_100 = trapz(f, 0.0, 1.0, 100)?;
    let trap_err_100 = (trap_100 - exact).abs();
    let trap_rel_100 = trap_err_100 / exact.abs();
    println!("{:<20} {:<15.8} {:<15.2e} {:<15.2e}", "Trapezoidal (100)", trap_100, trap_err_100, trap_rel_100);
    
    let trap_1000 = trapz(f, 0.0, 1.0, 1000)?;
    let trap_err_1000 = (trap_1000 - exact).abs();
    let trap_rel_1000 = trap_err_1000 / exact.abs();
    println!("{:<20} {:<15.8} {:<15.2e} {:<15.2e}", "Trapezoidal (1000)", trap_1000, trap_err_1000, trap_rel_1000);
    
    let simp_100 = simpson(f, 0.0, 1.0, 100)?;
    let simp_err_100 = (simp_100 - exact).abs();
    let simp_rel_100 = simp_err_100 / exact.abs();
    println!("{:<20} {:<15.8} {:<15.2e} {:<15.2e}", "Simpson (100)", simp_100, simp_err_100, simp_rel_100);
    
    let simp_1000 = simpson(f, 0.0, 1.0, 1000)?;
    let simp_err_1000 = (simp_1000 - exact).abs();
    let simp_rel_1000 = simp_err_1000 / exact.abs();
    println!("{:<20} {:<15.8} {:<15.2e} {:<15.2e}", "Simpson (1000)", simp_1000, simp_err_1000, simp_rel_1000);
    
    let romberg_6 = romberg(f, 0.0, 1.0, 1e-6, 20)?; // Add max_iterations parameter
    let romberg_err_6 = (romberg_6 - exact).abs();
    let romberg_rel_6 = romberg_err_6 / exact.abs();
    println!("{:<20} {:<15.8} {:<15.2e} {:<15.2e}", "Romberg (1e-6)", romberg_6, romberg_err_6, romberg_rel_6);
    
    let romberg_10 = romberg(f, 0.0, 1.0, 1e-10, 20)?; // Add max_iterations parameter
    let romberg_err_10 = (romberg_10 - exact).abs();
    let romberg_rel_10 = romberg_err_10 / exact.abs();
    println!("{:<20} {:<15.8} {:<15.2e} {:<15.2e}", "Romberg (1e-10)", romberg_10, romberg_err_10, romberg_rel_10);
    
    println!();
    println!("Recommendations:");
    println!("• Use Trapezoidal for quick approximations and smooth functions");
    println!("• Use Simpson for better accuracy with moderate computational cost");
    println!("• Use Romberg for high precision requirements");
    println!("• Consider function smoothness when choosing intervals");
}

Integration Method Selection Guide:

Method               Result          Error           Relative Error 
Trapezoidal (100)    1.14779947      5.89e-6         5.13e-6        
Trapezoidal (1000)   1.14779363      5.89e-8         5.13e-8        
Simpson (100)        1.14779357      2.95e-11        2.57e-11       
Simpson (1000)       1.14779357      1.11e-15        9.67e-16       
Romberg (1e-6)       1.14779357      3.32e-10        2.90e-10       
Romberg (1e-10)      1.14779357      4.44e-16        3.87e-16       

Recommendations:
• Use Trapezoidal for quick approximations and smooth functions
• Use Simpson for better accuracy with moderate computational cost
• Use Romberg for high precision requirements
• Consider function smoothness when choosing intervals


()

## Summary

This notebook demonstrated:

1. **Trapezoidal Rule**: Simple, O(h²) accuracy, good for quick estimates
2. **Simpson's Rule**: Higher O(h⁴) accuracy with parabolic approximation
3. **Romberg Integration**: Adaptive method with Richardson extrapolation
4. **Convergence Analysis**: Understanding error behavior and method selection
5. **Performance Trade-offs**: Balancing accuracy and computational cost
6. **Practical Applications**: Real-world integration problems

### Key Takeaways:

- **Method Selection**: Choose based on required accuracy and computational budget
- **Function Properties**: Smooth functions converge faster with fewer intervals
- **Error Scaling**: Simpson's rule has O(h⁴) vs trapezoidal's O(h²) convergence
- **Adaptive Methods**: Romberg integration automatically refines for target accuracy
- **Oscillatory Functions**: Require sufficient sampling to capture behavior

### Ergonomic Macros Used:

- `vec64![...]` for creating vectors with natural syntax
- `range!(start => end, n)` for generating linearly spaced data
- Proper plotting API with `.line_with()`, `.scatter_with()`, and `.legend(true)`

### Best Practices:

1. Start with Simpson's rule for general-purpose integration
2. Use Romberg for high-precision requirements
3. Increase intervals for oscillatory or discontinuous functions
4. Monitor convergence to validate results
5. Consider analytical solutions when available for verification

### Next Steps:

- Explore multi-dimensional integration techniques
- Study Monte Carlo integration methods
- Learn about specialized quadrature rules (Gaussian, Chebyshev)
- Investigate adaptive mesh refinement strategies